# Получение информации (индексов) с космоснимков

In [ ]:
import os
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import ee

### Авторизация Google Earth Engine API

In [ ]:
ee.Authenticate()
ee.Initialize()

In [ ]:
# coordinates region of interest (lat, lon)
df_generated_points = pd.read_csv('generated_points.csv')

### Алгоритм получения данных с космоснимков
Для получения индексов необходимо: выбрать точки пробоотбора, выбрать косми-ческую программу (коллекцию), определить область (в метрах), по которой будет произ-водиться анализ, выбрать временной промежуток, выбрать необходимый тип Reducer. 
Из полученных параметров формируется API-запрос на сервис GEE. После обра-ботки, сервис возвращает полученные индексы. В дальнейшем полученные индексы должны быть использованы для обучения моделей. Но перед тем, как обучать модели, нужно их отфильтровать.

В качестве примера продемонстрирован алгоритм получения NDVI.


### NDVI
Вегетационный индекс использует свойства красного каналов (кото-рый поглощает растительность) и ближних инфракрасных каналов (которые сильно отражают растительность). Как следует из названия, он используется для мониторинга здоровья и силы растительности. Нормализованный относительный индекс растительности (NDVI) всегда колеблется от -1 до +1. Отрицательные значения указывают на наличие воды и влаги. Но высокие значения NDVI предполагают густой растительный покров. Данный индекс активно примется в сельском хозяйстве для контроля состояния посевов.


In [ ]:
def get_ndvi_index(df, image_collection, start_date, end_date):
    """
    Get the index of the generated points.
    """
    ndvi_val = []

    for i in tqdm(range(len(df))):
        lon = df['Longitude'][i]
        lat = df['Latitude'][i]
        point = (ee.Geometry.Point(lon, lat))

        l8 = ee.ImageCollection(image_collection)
    
        image = ee.Image(l8.filterBounds(point)
            .filterDate(start_date, end_date)
            .sort('CLOUD_COVER')
            .first())
    
        nir = image.select('B5')
        red = image.select('B4')
        ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
        val = (ndvi.reduceRegion(
                            reducer=ee.Reducer.mean(),
                            geometry=point,
                            scale=10))
        
        try:
            ndvi_val.append(val.getInfo().get('NDVI'))
        except:
            ndvi_val.append(np.nan)
            continue
        
    return pd.Series(ndvi_val, copy=False)

In [ ]:
image_collection = 'LANDSAT/LC08/C01/T1_TOA'
start_date = '2015-01-01'
end_date = '2015-12-31'

In [ ]:
df_ndvi = get_ndvi_index(df_generated_points, image_collection, start_date, end_date)